                                             Reviews-Mumbai

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from selenium.common.exceptions import NoSuchElementException
import sys

In [2]:

#header
header={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36'}

## Function to get the webpage

In [3]:
def Search(search_query):
    url='https://www.practo.com/search?results_type=doctor&q=%5B%7B%22word%22%3A%22general%20physician%22%2C%22autocompleted%22%3Atrue%2C%22category%22%3A%22subspeciality%22%7D%5D&city=Mumbai&page='+search_query
    time.sleep(1)
    page=requests.get(url,headers=header)
    if page.status_code==200: 
        return page
    else:
        return "Error"

## Function to scrape the link of individual doctor's profile

In [4]:
link=[]
for i in range(80):   #Doctor profiles are in 80 pages
    search_response=Search(str(i))
    soup=BeautifulSoup(search_response.content)#soup has all the content in the page
    links= soup.findAll('a',{'class':"u-color--primary"}) 
    for l in links:
        link.append(l.get('href'))#links are saved in a list

In [5]:
df=pd.DataFrame()

df['link']=link #converting the list to dataframe

## Function to change the address of the link(so that we will get the recent reviews)

In [6]:
def clean(row):
    Link=row['link']
    Link=Link.split('?')
    return Link[0]+'/recent'

In [7]:
df['link']=df.apply(lambda row:clean(row),axis=1) #function calling

In [8]:
df=df.drop_duplicates(subset='link').reset_index(drop=True) #dropping all duplicate links

In [9]:
Links=df['link']

## Initializing selenium 

###### As we have infinite scrolling, selenium will be easy to click the more button

In [10]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(executable_path='C:/Users/VIJESH\Desktop/chromedriver.exe')

## Function to click more button (to get all the reviews of the respective doctor)

In [17]:
def review_page(query):    
    driver.get("https://www.practo.com"+Links[query]) #opens the profile of the doctors
    for i in range(45):                                                           #45 times clicks the more reviews button
        try:
            driver.find_element_by_xpath('//button[@data-qa-id="view-more-feedback"]').click() 
            time.sleep(2)
        except NoSuchElementException: 
            pass
    return driver.page_source  #returns the page

## Function to scrape the reviews

In [18]:
review=[]
rev=''
recom=[]
import time
for i in range(len(Links)): 
    soup = BeautifulSoup(review_page(int(i)), 'lxml')#soup has the content of the doctor's profile
    for rev_clas in soup.findAll(attrs={'class':"feedback__body"}): #accessing the tad in which reviews are present
        for i in rev_clas.findAll(attrs={'data-qa-id':"review-text"}):#saves all the patient reviews 
            rev=rev+i.text
        review.append(rev)
        rev=''
    for rev_clas in soup.findAll(attrs={'class':"pure-g feedback--item u-cushion--medium-vertical"}):
        for i in rev_clas.findAll(attrs={'class':'u-cushion--small-bottom u-large-font'}): #saves the recommendations
            recom.append(i.text)

In [19]:
len(review) #total length

2296

In [28]:
len(recom) #total recommendations

2296

In [29]:
df_1=pd.DataFrame()

## creating a dataframe

In [30]:
df_1['reviews']=review
df_1['recommendation']=recom 

In [31]:
df_1=df_1.drop_duplicates() #dropping all duplicates

## Function to tag the location

In [32]:
def location(row):
    each_row=row['reviews']
    return 'Mumbai'

In [33]:
df_1['location']=df_1.apply(lambda row:location(row),axis=1) #calling function

In [34]:
df_1.head()

,reviews,recommendation,location
0,"Dr Sanjiv Shah is a very intelligent, thorough...",I recommend the doctor,Mumbai
1,Doctor is result oriented and I am very satisf...,I recommend the doctor,Mumbai
2,Dr Sanjiv Shah is a renouned diabatologist and...,I recommend the doctor,Mumbai
3,"I only trust him with my thyroid treatment , e...",I recommend the doctor,Mumbai
4,Dr Sanjiv Shah was very friendly.He explained ...,I recommend the doctor,Mumbai


In [35]:
df_1.to_csv('doctor_Mumbai.csv',index=False) #saving as CSV